In [42]:
import numpy as np
from PIL import Image
from pydub import AudioSegment
from pydub.generators import Sine

def load_image(image_path, target_resolution=(1920, 1080)):
    # Load and resize the image to the target resolution
    image = Image.open(image_path).convert('L')
    image = image.resize(target_resolution, Image.Resampling.LANCZOS)
    image_data = np.array(image)
    return image_data

def map_brightness_to_frequency(image_data, min_freq=220, max_freq=880):
    # Normalize pixel values to range from 0 to 1 and map to frequency range
    norm_image_data = image_data / 255.0
    frequencies = min_freq + (max_freq - min_freq) * norm_image_data
    return frequencies

def generate_sound_with_pydub(frequencies, duration_per_tone=5, sample_rate=44100):
    # Generate an audio signal using pydub for a subset of frequencies
    h, w = frequencies.shape
    audio_signal = AudioSegment.silent(duration=0)

    # Subsample image data to reduce the number of tones
    stride = max(1, int(np.sqrt((h * w) / ((sample_rate // 1000) * duration_per_tone * 2))))  # Stride based on desired audio length
    subsampled_frequencies = frequencies[::stride, ::stride]

    # Generate sound for each subsampled frequency
    for freq in subsampled_frequencies.flatten():
        tone = Sine(freq).to_audio_segment(duration=duration_per_tone)
        audio_signal += tone
    
    return audio_signal

def save_sound_to_wav(audio_signal, output_file='output_sound2.wav'):
    # Export the audio signal as a WAV file
    audio_signal.export(output_file, format="wav")

# Example usage
image_path = 'space.jpg'
image_data = load_image(image_path, target_resolution=(1920, 1080))
frequencies = map_brightness_to_frequency(image_data)

# Set duration_per_tone to achieve a total audio length between 30s and 2min
# Adjusting stride and duration_per_tone will help fine-tune the length
duration_per_tone = 25  # Duration per tone in milliseconds

audio_signal = generate_sound_with_pydub(frequencies, duration_per_tone=duration_per_tone)
save_sound_to_wav(audio_signal)
